**In this notebook, I would like to predicting power fails in 10 days periods.**
===========================================
1. Generate a dataset for this task
2. Try out and evaluate models

In [3]:
#Import packages
import pytz
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import re
from IPython.display import Image
import itertools
import folium
import pandas as pd
import folium
from IPython.display import display
import geojson

In [4]:
#Set up Working Directory and Data Files
drive = 'G'
if drive == 'G':
    data_dir = '../01.data/'
    output_dir = '../05.outputs/'
else:
    data_dir = '..\\01.data\\'


In [6]:
file_sms_v2="/Users/dmatekenya/Google Drive/World-Bank/electricity_monitoring/01.data/sms_v2_2017-08-06.csv"
cols_to_use = ['box_id', 'psu', 'lon', 'lat', 'datetime_sent_raw', 'str_datetime_sent',
                   'str_datetime_sent_hr', 'hour_sent', 'event_type_str','event_type_num',
                   'power_state','data_source']

df = pd.read_csv(file_sms_v2, usecols=cols_to_use, parse_dates=['str_datetime_sent_hr'], nrows=10000)

In [8]:
df.event_type_str.value_counts()

pon_mon      5237
missing      1827
pfail        1303
pback        1158
pfail_mon     449
test           26
Name: event_type_str, dtype: int64

In [ ]:
def  poweroutage_counts (df_power_outage,df_box,time_intervals):
    """
    This function computes number of outages per unit of time
    for all the events which happened when the boxes were functioning.
    :param power_outage_source: csv file of power outages processed by JP
    :param time_intervals: e.g. daily
    :returns a pd.DataFrame object with the following columns: BoxID, lat/lon, time_unit, count
    """

    if time_intervals == 'daily':
        df = df_power_outage.groupby(['BoxID','date_powerfailure'])['POWERout'].count()
        by_box = df.reset_index()
    
        #merge the two datasets
        by_box2 = pd.merge (left = by_box, right=bx, on='BoxID', how='left')
        #powerout count by location
        by_box3 = by_box2.groupby(['lon','lat','date_powerfailure'])['POWERout'].count()
        by_latlon = by_box3.reset_index()
        
        #prettify the DF
        by_latlon.rename(columns={'date_powerfailure': 'date', 'POWERout': 'outage_freq'}, inplace=True)
        
        return by_latlon

In [ ]:
def preprocess_powerout_data(file_name):
    """
    Does some data wrangling for use in the prediction of average duration
    :param file_name: csv file of power outages processed by JP
    :returns a pd.DataFrame object
    """
    # Load data of hourly power events: data prpcessed by JP
    df = pd.read_csv(file_name)

    # Kepp only required columns
    to_keep = ['BoxID', 'date_powerfailure', 'date_powerfailure_hour',
               'date_powerback', 'date_powerback_hour', 'duration', 'POWERout']

    d = df[to_keep]
    d.is_copy = False

    # Change duration to hours
    d['duration_hrs'] = d['duration'] / 60

    # consider only valid power out events
    d = d[d['duration_hrs'].notnull()]

    # change to datetime objects
    d['date_powerfailure2'] = d.apply(lambda x: pd.datetime.strptime(x['date_powerfailure'], '%d%b%Y'), axis=1)

    d.drop('date_powerfailure', axis=1, inplace=True)

    d.rename(columns={'date_powerfailure2': 'date_powerfailure'}, inplace=True)

    return d

In [ ]:
d = preprocess_powerout_data(data_dir + 'powerout.csv')

In [ ]:
d_d = d.groupby(['BoxID','date_powerfailure'])['POWERout'].count()
by_box = d_d.reset_index()

In [ ]:
df = pd.DataFrame({'date': pd.date_range (d.date_powerfailure.min(),d.date_powerfailure.max())})
df['outage_freq'] = 0

In [ ]:
by_latlon_full = pd.merge (left = df, right=by_latlon, left_on='date', right_on='date_powerfailure', how='right')

In [76]:
df.shape

(153, 2)

In [81]:
by_latlon['latlon'] = by_latlon.apply (lambda row: str(row['lon']) + ','+ str(row['lat']),axis=1)

In [75]:
pow

,lon,lat,date_powerfailure,POWERout,loc
0,38.533566,68.862146,2016-12-02,2,"38.53356644,68.86214562"
1,38.533566,68.862146,2016-12-03,2,"38.53356644,68.86214562"
2,38.533566,68.862146,2016-12-04,2,"38.53356644,68.86214562"
3,38.533566,68.862146,2016-12-06,2,"38.53356644,68.86214562"
4,38.533566,68.862146,2016-12-07,2,"38.53356644,68.86214562"


In [ ]:
#import box details
bx = pd.read_csv(data_dir + 'Boxes.csv', usecols=['LONG','LAT','ClusterId','BoxID'])
#rename some cols
bx.rename (columns={'LONG': 'lon','LAT':'lat'},inplace=True)

In [ ]:
#merge the two datasets
by_box2 = pd.merge (left = by_box, right=bx, on='BoxID', how='left')

In [119]:
uniq_locs = list (by_latlon.latlon.unique())
df_full = pd.DataFrame()
for l in uniq_locs:
    df_loc = by_latlon[by_latlon.latlon==l]
    merged= pd.merge (left = df, right=df_loc,left_on='date', right_on='date_powerfailure', how='left')
    merged ['outage_freq'] = merged.apply (lambda x: max (x['outage_freq'],x['POWERout']),axis=1)
    merged['latlon'] = l
    merged['lon'] = float(l.split(',')[1])
    merged['lat'] = float(l.split(',')[0])
    df_full = df_full.append(merged)
    

In [121]:
df_full

,date,outage_freq,lon,lat,date_powerfailure,POWERout,loc,latlon
0,2016-10-22,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
1,2016-10-23,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
2,2016-10-24,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
3,2016-10-25,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
4,2016-10-26,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
5,2016-10-27,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
6,2016-10-28,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
7,2016-10-29,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
8,2016-10-30,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"
9,2016-10-31,0.0,68.862146,38.533566,NaT,NaN,NaN,"38.53356644,68.86214562"


In [93]:
df_loc = by_latlon[by_latlon.latlon==uniq_locs[0]]

In [96]:
df_full = pd.DataFrame()
merged =pd.merge (left = df, right=df_loc,left_on='date', right_on='date_powerfailure', how='left')

In [105]:
merged ['outage_freq'] = merged.apply (lambda x: max (x['outage_freq'],x['POWERout']),axis=1)
merged['latlon'] = uniq_locs[0]
merged['lon'] = float(l.split(',')[1])
merged['lat'] = float(l.split(',')[0])

In [112]:
float(uniq_locs[0].split(',')[0])

38.53356644

In [ ]:
by_latlon.POWERout.value_counts()

In [113]:
uniq_locs[0]

'38.53356644,68.86214562'

In [ ]:
d = preprocess_powerout_data(data_dir + 'powerout.csv')

In [ ]:
d_d = d.groupby(['BoxID','date_powerfailure'])['POWERout'].count()

by_box = d_d.reset_index()

#add box locaiton details
#import box details
bx = pd.read_csv(data_dir + 'Boxes.csv', usecols=['LONG','LAT','ClusterId','BoxID'])
#rename some cols
bx.rename (columns={'LONG': 'lon','LAT':'lat'},inplace=True)
    
#merge the two datasets
by_box2 = pd.merge (left = by_box, right=bx, on='BoxID', how='left')

#powerout count by location
df = by_box2.groupby(['lon','lat','date_powerfailure'])['POWERout'].count().reset_index()

In [ ]:
foo = df.ix[(df['lon']==38.533566) & (df['lat'] == 68.862146)]
foo.head()

In [ ]:
df.head()

In [ ]:
def compute_mean_duration (df, cut_off_date):
    '''
    The idea is to track number of hours when power is off not
    later than some given cut off date by box.
    @param:  df: the data file under consideration
    @param:  cut_off_date: upper date limit for considerin events
    @return: a pd.DataFrame object containing average duration of each box
    '''
    #select events earlier than cut off date
    d_date = df[df.date_powerfailure <= cut_off_date]
    
    #select only powerout events
    d_out = d_date[d_date.duration_hrs.notnull()]
    
    #Exclude all outages longer than 1 day
    print ('Leaving out %s events whose power out duration is over 24 hrs!!!' %(len(d_out[d_out.duration_hrs>=24])))
    
    d_out = d_out[d_out.duration_hrs < 24]
    
    #Group by box id and compute mean
    by_box = d_out.groupby(['BoxID'])['duration_hrs'].mean()
    by_box = by_box.reset_index()
    
    #add box locaiton details
    #import box details
    bx = pd.read_csv(data_dir + 'Boxes.csv', usecols=['LONG','LAT','ClusterId','BoxID'])
    #rename some cols
    bx.rename (columns={'LONG': 'lon','LAT':'lat'},inplace=True)
    
    #merge the two datasets
    by_box2 = pd.merge (left = by_box, right=bx, on='BoxID', how='left')

    #merge again by psu-lat/lon
    by_xy = by_box2.groupby(['ClusterId','lon','lat'])['duration_hrs'].mean()
    
    print ('Completed adding mean to df.....')
    
    return by_xy.reset_index()

In [ ]:
by_xy = compute_mean_duration (d,date)

In [ ]:
by_xy.shape

In [ ]:
date

In [ ]:
date = d.date_powerfailure.max()

d_date = d[d <= date]

d_out = d[d.duration_hrs.notnull()]
    
d_out = d_out[d_out.duration_hrs < 24]

by_box = d_out.groupby(['BoxID'])['duration_hrs'].mean()

In [ ]:
by_box = by_box.reset_index()

In [ ]:
by_box2 = pd.merge (left = by_box, right=bx, on='BoxID', how='left')

In [ ]:
by_box2.shape

In [ ]:
by_box2 [by_box2.ClusterId==1]

In [ ]:
by_loc=by_box2.groupby(['ClusterId','lon','lat'])['duration_hrs'].mean()

In [ ]:
np.array([0.977412,0.481998]).mean()

In [ ]:
def data2geojson(df):
    features = []
    df.apply(lambda X: features.append( 
            geojson.Feature(geometry=geojson.Point((X["lat"], 
                                                    X["lon"])), 
                properties=dict(name=X["ClusterId"], 
                                description=X["avg_duration"]))
                                    )
            , axis=1)
    
    return features

In [ ]:
feat = data2geojson(by_loc)

In [ ]:
df_out = by_loc.reset_index()

In [ ]:
df_out.to_csv (data_dir + 'powerout_avg_duration_by_loc')

In [ ]:
#Kepp only required columns
to_keep = ['BoxID', 'date_powerfailure', 'date_powerfailure_hour',
       'date_powerback', 'date_powerback_hour','duration','POWERout']

d = d[to_keep]

#Change duration to hours
d['duration_hrs'] = d.apply(lambda x: x['duration']/60,axis=1)

#change to datetime objects
d['date_powerfailure'] = d.apply(lambda x: pd.datetime.strptime(x['date_powerfailure'], '%d%b%Y'), axis=1)

In [ ]:
str = ''

if str is None:
    print ('None')

In [ ]:
d_out.h

In [ ]:
def compute_mean_duration (df, cut_off_date):
    '''
    The idea is to track number of hours when power is off not
    later than some given cut off date by box.
    @param:  df: the data file under consideration
    @param:  cut_off_date: upper date limit for considerin events
    @return: a pd.DataFrame object with two columns containing average duration of each box
    '''
    #select events earlier than cut off date
    d_date = d[d <= date]
    
    #selec only powerout events
    d_out = d[d.duration_hrs.notnull()]
    
    #Exclude all outages longer than 2 days ()
    print ('Leaving out %s events whose power out duration is over 24 hrs!!!' %(len(d_out[d_out.duration_hrs>=24])))
    d_out = d_out[d_out.duration_hrs < 24]
    #Group by box id and compute mean
    by_box = d_out.groupby(['BoxID'])['duration_hrs'].mean()
    
    return by_box.reset_index()

In [ ]:
d.head()

In [ ]:
d2 = compute_mean_duration (d, '2016-10-30')

In [ ]:
d2.head()

In [ ]:
#select events earlier than cut off date
date = pd.Timestamp('2016-10-30')

d_date = d[d.date_powerfailure <= date]
    
    #selec only powerout events
d_out = d[d.duration_hrs.notnull()]
    
#Exclude all outages longer than 2 days ()
    
print ('Leaving out %s ' %(len(d_out[d_out.duration_hrs>=24])))
    
#Group by box id and compute mean
    
by_box = d_out.groupby(['BoxID'])['duration_hrs'].mean()

return by_box.reset_index()

**Predict whether there was a power failure in the past 1 day**
================================================================

In [ ]:
#Collapse data to daily level
#For all the hours of the day, I simply take the maximum POWERout val
d_dy = pd.DataFrame(d.groupby(['BoxID','date_powerfailure'])['POWERout'].max())
d_dy.reset_index(inplace=True)
d_dy.rename(columns={'POWERout': 'pwrout_1dy'},inplace=True)

In [ ]:
d_dy.head()

In [ ]:
def generate_train_data (d_raw,date,num_days,target):
    """
    This function returns a training dataset for prediction.

    @param d_raw: reasonably preprocessed data
    @param date: date threshold (when to stop) e.g., 2016/10/20
    @param num_days: Number of days to consider
    @return: Returns a dataset where each row represents a time interval(num_days) with corresponding
    @raise keyError: raises an exception
    """
    
    #create a list of num_days time intervals covering 2016 october-1 up date
    start = d_raw.date_powerfailure.min().date()
    end = datetime.strptime(date,'%Y/%m/%d').date()
    delta = timedelta(days=num_days)
    date_list = []

    while start <= end:
        tup = (start,start+delta)
        date_list.append (tup)

        start +=  timedelta(days=num_days+ 1)
        
    #Use cartesian product to create a dataframe of the dates created above
    #and boxID's
    date_boxid = pd.DataFrame(list(itertools.product(date_list,list(d_raw.BoxID.unique()))),columns=['dates','BoxID'])
    
    result =[]
    d_10dy = pd.DataFrame
    for date in list(date_list):
        #Select events in this range
        mask = (d_raw['date_powerfailure'] >= date[0]) & (d_raw['date_powerfailure'] <= date[1])
        d_raw_within = d_raw.loc[mask]
        
        #Summarise powerouts to num_days level
        d_raw_by_box = pd.DataFrame(d_raw_within.groupby(['BoxID'])[target].max())
        d_raw_by_box.reset_index(inplace=True)
        
        new_powerout_col = 'pwrout_' + str(num_days) + 'dy'
        d_raw_by_box.rename(columns={target: new_powerout_col},inplace=True)
        d_raw_by_box['dates'] = [date for i in range(d_raw_by_box.shape[0])]
        
        #create a df of summarised power failure events
        if len (result) == 0:
            d_10dy = d_raw_by_box
        else:
            d_10dy = d_10dy.append (d_raw_by_box)
        
        result.append (d_raw_by_box)
            
    #print (date_boxid.head())
    d2 = pd.merge (left = bx, right=d_10dy,on='BoxID', how='right')
    
    #Rearrange columns
    d2 = d2 [['dates','ClusterId','BoxID','LONG','LAT','pwrout_10dy']]
    
    #rename soem cols
    d2.rename (columns={'LONG': 'lon','LAT':'lat'},inplace=True)
    return d2

In [ ]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance (pt1,pt2):
    """
    Computes distance between two geographic coordinates
    :param pt1: [Lat,Lon] for first point
    :param pt2: [Lat,Lon] for second
    :returns distance in km between the two points
    """
    # Radius of the earth in km (Hayford-Ellipsoid)
    EARTH_RADIUS = 6378388/1000
    
    d_lat = radians (pt1[0] - pt2[0])
    d_lon = radians (pt1[1] - pt2[1])
    
    lat1 = radians(pt1[0])
    lat2 = radians(pt2[0])
    
    a = sin(d_lat/2) * sin(d_lat/2) + \
        sin(d_lon/2) * sin(d_lon/2) * cos(lat1) * cos(lat2)
        
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    return c * EARTH_RADIUS

In [ ]:
def predict_with_nearest_neighbor (poi,points,target):
    """
    Computes distance between a point of interest and the rest of the points
    :param pt1: [Lat,Lon] for first point
    :param pt2: [Lat,Lon] for second
    :returns a dict object with BoxID and distances
    """
    #dict object to 
    distances = {}
    min = 500000.000
    min_bx = ''
    predicted_pwrstate = ''
    
    for bx in list(points.BoxID.unique()):
        d_bx = points[points.BoxID == bx]
        dist = calculate_distance (poi,[d_bx.lat,d_bx.lon])
        
        if dist < min:
            min = dist
            predicted_pwrstate = d_bx[target].values[0]
            min_bx = bx
    
    #print (predicted_pwrstate)
    return [min_bx, predicted_pwrstate, min]

In [ ]:
#Inorder to get location of boxes, I merge the d_dy dataframe with boxes details
d3 = pd.merge (left = d2, right=bx, on='BoxID', how='left')

In [ ]:
d3.head()

In [ ]:
def predict_with_nearest_neighbor2(df, target, num_neighbors, how):
    """
    Given a dataframe with lat/lon and average duration of power failure for that location
    this function finds the nearest point to the target point and return the avg power duration
    from the nearest point.
    :param df: pd.DataFrame with cols: BoxID, lat/lon, avg power failure duration (hours) 
    :param target: [lat,lon]-The unsampled location where the avg power failure duration needs to be estimated
    :param num_neighbors: NUmber of neighbors to use for the nearest neigbor
    :param how: The approach to use. If num_neighbors == 1, just return value of nearest point..otherwise, it can
    be 'mean', 'mode' or 'median'.
    :returns the avg power failure duration and posssibly the BoxID
    """
    #Just incase, remove all duplicate lat/lon pairs
    df = df.drop_duplicates (['lon','lat'])
    
    df ['dist'] = df.apply(lambda row: calculate_distance([row['lat'],row['lon']],target),axis=1)
    
    #In the simplest case, assign to this new location valaue based on a single nearest neighbor
    if num_neighbors==1:
        min_dist = df.dist.min()
        predicted = df[df.dist==min_dist].duration_hrs.values[0]
        return predicted
    else: #cases with more than one neighbor
        #Select only the 'num_neaighbors' nearest neighbors
        df = df.sort_values (by = ['dist'], ascending=True)[:num_neighbors]
        if how=='wmean':
            #Add weight based on distance
            df['dist_wght'] = 1/df['dist']
            
            #compute simple weighted mean
            wmean = (df['duration_hrs'] * df['dist_wght']).sum() /df.dist_wght.sum()
            
            return wmean
        elif how == 'mean':
            return df['duration_hrs'].mean (axis=0)
        elif how == 'median':
            return df['duration_hrs'].median (axis=0)


In [ ]:
pred_nearest = predict_with_nearest_neighbor2(d3,[38.694671,69.600202],1,'mean')
pred_nearest

In [ ]:
pred_5_mean = predict_with_nearest_neighbor2(d3,[38.694671,69.600202],5,'mean')
pred_5_mean

In [ ]:
pred_5_wmean = predict_with_nearest_neighbor2(d3,[38.694671,69.600202],5,'wmean')
pred_5_wmean

In [ ]:
d3[d3.dist==min_dist]

In [ ]:
def simple_mean (df, idw):
    #add weighted duration-weights is 1/distance
    df['dist_wght'] = 1/df['dist']
    
    #compute mean based on weighted duration
    mean = df.['duration_hrs_wgted'].mean()
    
    if idw:
        wmean = (df['duration_hrs'] * df['dist_wght']).sum() /df.dist_wght.sum()
        
        return wmean

In [ ]:
df['duration_hrs_wgted'] = df.apply (lambda x: 1/x['dist'] * x['duration_hrs'],axis=1)

In [ ]:
df.duration_hrs_wgted.mean(axis = 0)

In [ ]:
df['dist_wght'] = 1/df['dist']

(df['duration_hrs'] * df['dist_wght']).sum() /df.dist_wght.sum()

In [ ]:
df.duration_hrs.mean(axis = 0)

**Finally lets see perfomance on separate locations....**
============================================================
Note that this evaluation is being done on sampled locations...just leaving them out

In [ ]:
#Inorder to get location of boxes, I merge the d_dy dataframe with boxes details
d2 = pd.merge (left = d_dy, right=bx, on='BoxID', how='left')

In [ ]:
#Export this data for quick demo
d2.to_csv('../01.data/powerouts_1dy.csv',index=False)

In [ ]:
#A batch Evaluation
evaluate_nearest_neighbor_predictor(d2, 50, 50)

In [ ]:
folium?

**Make predictions on random locaitons on a map**
===================================================

In [ ]:
def data2geojson2(df):
    features = []
    df.apply(lambda X: features.append( 
            geojson.Feature(geometry=geojson.Point((X["lon"], 
                                                    X["lat"])), 
                properties=dict(ClusterID=X["ClusterId"], 
                                avg_duration=X["duration_hrs"]))
                                    )
            , axis=1)
    
    return features

In [ ]:
f = data2geojson2 (by_loc.reset_index())

In [ ]:
f[0].properties['avg_duration']

In [ ]:
coords = d2 [['lon','lat', 'ClusterId', 'BoxID']]
coords = coords.sort_values(by  = ['lon','lat'] )
coords['xy'] = coords.apply(lambda x: (x['lon'], x['lat']),axis=1)
xy = coords.xy.unique()

In [ ]:
coords.head()

In [ ]:
j = coords.to_json()

In [ ]:
j.lon

In [ ]:
print ('Predicted power state-%s  from box at distance %s km '%(predicted[1],predicted[2]))

In [ ]:
boxid = predicted[0]

box_loc = [d2[d2.BoxID==boxid].lon,d2[d2.BoxID==boxid].lat]

In [ ]:
xy = pd.read_csv('../01.data/powerouts_1dy.csv')
	
features = data2geojson2(xy)

In [ ]:
def add_boxes_location():
    """Get location of the boxes and them to the map"""
    features = []
    df = pd.read_csv(data_path + 'powerouts_1dy.csv')
    df.apply(lambda X: features.append( 
            geojson.Feature(geometry=geojson.Point((X["lon"],X["lat"])), 
            properties=dict(name=X["BoxID"], description=X["ClusterId"])))
            , axis=1)
    
    return geojson.dumps(geojson.FeatureCollection(features), sort_keys=True)


In [ ]:
df.head()